In [ ]:
# Basic RAG with FAISS + HuggingFace LLM

In [1]:
!pip install -q faiss-cpu sentence-transformers transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.2 MB/s eta 0:00:00


In [2]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
import numpy as np

In [3]:
# Load embedding model
embedder = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# Sample documents
docs = [
    "The Eiffel Tower is in Paris.",
    "The Great Wall of China is visible from space.",
    "Pandas are native to China.",
    "The Taj Mahal is in India.",
    "Python is a programming language.",
    "Vidiyal sessions are conducted in Coimbatore."
]

# Create document embeddings
doc_embeddings = embedder.encode(docs)

In [5]:
# Create FAISS index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)

In [6]:
# User query
query = "Where is the Eiffel Tower?"
query_embedding = embedder.encode([query])

In [10]:
# Search
D, I = index.search(query_embedding, k=3)
matches = [docs[i] for i in I[0]]

In [11]:
# Display matches
print("Query:", query)
print("Top Documents:")
for match in matches:
    print("-", match)

Query: Where is the Eiffel Tower?
Top Documents:
- The Eiffel Tower is in Paris.
- The Great Wall of China is visible from space.
- The Taj Mahal is in India.


In [12]:
# Load text generation model
qa_model = pipeline("text2text-generation", model="google/flan-t5-base")

# Combine matched docs with query
context = " ".join(matches)
prompt = f"Answer based on context: {context} Question: {query}"

Device set to use cpu


In [13]:
# Generate answer
response = qa_model(prompt, max_length=100)[0]['generated_text']
print("\nAnswer:", response)


Answer: Paris


In [ ]:
# User query
query1 = "Where is the Vidiyal session conducted?"
query_embedding1 = embedder.encode([query1])

# Search
D1, I1 = index.search(query_embedding1, k=2)
matches1 = [docs[i] for i in I1[0]]

# Display matches
print("Query:", query1)
print("Top Documents:")
for match in matches1:
    print("-", match)

# Load text generation model
qa_model1 = pipeline("text2text-generation", model="google/flan-t5-base")

# Combine matched docs with query
context = " ".join(matches1)
prompt1 = f"Answer based on context: {context} Question: {query1}"

# Generate answer
response1 = qa_model(prompt1, max_length=100)[0]['generated_text']
print("\nAnswer:", response1)

Query: Where is the Vidiyal session conducted?
Top Documents:
- Vidiyal sessions are in Coimbatore.
- The Taj Mahal is in India.


Device set to use cpu



Answer: Coimbatore
